### Example program

- From *Kernel-based Detection of Coincidentally Correct Test Cases to Improve Fault Localization Effectiveness*

```c
 e1: int Sample(int a, int b, int c) {
 e2:    int rsum, rdiv, result, rlog=0;
 e3:    result = 0;
 e4:    rdiv = 1;
 e5:    rsum = a + b;
 e6:    if ((a > 0) && (b > 0))
 e7:        rdiv = a / b;
 e8:    rmax = b;
 e9:    if (a > b)
e10:        rmax = b; // Correct: rmax = a;
e11:    if (c == 1)
e12:        result = rsum;
e13:    if (c == 2)
e14:        result = rdiv;
e15:    if (c == 3)
e16:        result = rmax;
e17:    return result;
     }
```

# SBFL on example program

In [15]:
import pandas as pd

idxs = [
    "e6",
    "e7",
    "e8",
    "e9",
    "e10",
    "e11",
    "e12",
    "e13",
    "e14",
    "e15",
    "e16",
    "e17",
]

exec1_et = [0, 1, 2, 3, 4, 5, 7, 9, 10, 11]
exec2_et = [0, 2, 3, 4, 5, 7, 8, 9, 11]
exec2_et_2 = [0, 2, 3, 4, 5, 7, 8, 9, 11]
exec3_et = [0, 2, 3, 5, 7, 9, 10, 11]
exec4_et = [0, 1, 2, 3, 4, 5, 6, 7, 9, 11]
exec5_et = [0, 1, 2, 3, 5, 7, 9, 10, 11]

execs = [exec1_et, exec2_et, exec2_et_2, exec3_et, exec4_et, exec5_et]

results = [1, 0, 0, 0, 0, 0]
op2s = []
for i in range(12):
    ep, ef = 0, 0
    for exec, result in zip(execs, results):
        if i in exec:
            if result:
                ef += 1
            else:
                ep += 1
    op2 = ef - ep / (len(results) - sum(results) + 1)
    op2s.append(op2)

df = pd.DataFrame({"idx": idxs, "op2": op2s})
df["rank"] = df.op2.rank(ascending=False)
df.sort_values(by="rank", inplace=True)
df


,idx,op2,rank
1,e7,0.666667,1.5
10,e16,0.666667,1.5
4,e10,0.500000,3.0
0,e6,0.166667,7.0
2,e8,0.166667,7.0
3,e9,0.166667,7.0
5,e11,0.166667,7.0
7,e13,0.166667,7.0
9,e15,0.166667,7.0
11,e17,0.166667,7.0


# Create a factor graph of example program

In [16]:
%load_ext autoreload
%autoreload 2

from factorgraph import FactorNode, VariableNode, FactorGraph

variable_list = [VariableNode(idx) for idx in idxs]
exec_names = [    
    "exec1_et",
    "exec2_et",
    "exec2_et_2",
    "exec3_et",
    "exec4_et",
    "exec5_et",
]
for exec_name, result in zip(exec_names, results):
    testnode = VariableNode(exec_name)
    testnode.val = result
    testnode.fix()
    variable_list.append(testnode)

def prob_succ(elem_faulty):
    # This one is ad-hoc one.
    num_f = sum(elem_faulty)
    num_c = len(elem_faulty) - num_f
    return 0.01 ** (num_f / 1.1 ** num_c) 

def prob_match(result, elem_faulty):
    return 1 - prob_succ(elem_faulty) if result else prob_succ(elem_faulty)

scope = {"prob_succ": prob_succ, "prob_match": prob_match}

factor_list = []
for exec_name, et in zip(exec_names, execs):
    covered_elems = [idxs[i] for i in et]
    param_str = ", ".join([f"V[{elem}]" for elem in covered_elems])
    factor_node = FactorNode(
        exec_name, f"prob_match(V[{exec_name}], [{param_str}])")
    factor_list.append(factor_node)


factor_graph = FactorGraph(variable_list, factor_list)
print(factor_graph)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
VariableNode(name=e6, val=None, fixed=False, prob=None)
VariableNode(name=e7, val=None, fixed=False, prob=None)
VariableNode(name=e8, val=None, fixed=False, prob=None)
VariableNode(name=e9, val=None, fixed=False, prob=None)
VariableNode(name=e10, val=None, fixed=False, prob=None)
VariableNode(name=e11, val=None, fixed=False, prob=None)
VariableNode(name=e12, val=None, fixed=False, prob=None)
VariableNode(name=e13, val=None, fixed=False, prob=None)
VariableNode(name=e14, val=None, fixed=False, prob=None)
VariableNode(name=e15, val=None, fixed=False, prob=None)
VariableNode(name=e16, val=None, fixed=False, prob=None)
VariableNode(name=e17, val=None, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None

# Infer the random values that maximize the probability

In [22]:
factor_graph.inference_exhaustive(scope)
print(factor_graph)

inference_exhaustive
num cases: 4096
VariableNode(name=e6, val=0, fixed=False, prob=None)49973297119145
VariableNode(name=e7, val=1, fixed=False, prob=None)
VariableNode(name=e8, val=0, fixed=False, prob=None)
VariableNode(name=e9, val=0, fixed=False, prob=None)
VariableNode(name=e10, val=0, fixed=False, prob=None)
VariableNode(name=e11, val=0, fixed=False, prob=None)
VariableNode(name=e12, val=0, fixed=False, prob=None)
VariableNode(name=e13, val=0, fixed=False, prob=None)
VariableNode(name=e14, val=0, fixed=False, prob=None)
VariableNode(name=e15, val=0, fixed=False, prob=None)
VariableNode(name=e16, val=0, fixed=False, prob=None)
VariableNode(name=e17, val=0, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(

e7 is nominated as a buggy element, but it is not a bug.
Therefore, set e7 not buggy.

In [4]:
v_e7 = factor_graph.get_variable("e7")
v_e7.val = 0
v_e7.fix()
print(v_e7)

VariableNode(name=e7, val=0, fixed=True, prob=None)


Do inference again

In [5]:
factor_graph.reset()
factor_graph.inference_exhaustive(scope)
print(factor_graph)

VariableNode(name=e6, val=0, fixed=False, prob=None)
VariableNode(name=e7, val=0, fixed=True, prob=None)
VariableNode(name=e8, val=0, fixed=False, prob=None)
VariableNode(name=e9, val=0, fixed=False, prob=None)
VariableNode(name=e10, val=0, fixed=False, prob=None)
VariableNode(name=e11, val=0, fixed=False, prob=None)
VariableNode(name=e12, val=0, fixed=False, prob=None)
VariableNode(name=e13, val=0, fixed=False, prob=None)
VariableNode(name=e14, val=0, fixed=False, prob=None)
VariableNode(name=e15, val=0, fixed=False, prob=None)
VariableNode(name=e16, val=1, fixed=False, prob=None)
VariableNode(name=e17, val=0, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(name=exec5_et, val=0, fixed=True, prob=None)
FactorN

Then, e16 is nominated as a buggy element, but it is neither a bug.
Again, set e16 not buggy.

In [6]:
v_e16 = factor_graph.get_variable("e16")
v_e16.val = 0
v_e16.fix()
print(v_e16)

VariableNode(name=e16, val=0, fixed=True, prob=None)


In [7]:
factor_graph.reset()
factor_graph.inference_exhaustive(scope)
print(factor_graph)

VariableNode(name=e6, val=0, fixed=False, prob=None)
VariableNode(name=e7, val=0, fixed=True, prob=None)
VariableNode(name=e8, val=0, fixed=False, prob=None)
VariableNode(name=e9, val=0, fixed=False, prob=None)
VariableNode(name=e10, val=1, fixed=False, prob=None)
VariableNode(name=e11, val=0, fixed=False, prob=None)
VariableNode(name=e12, val=0, fixed=False, prob=None)
VariableNode(name=e13, val=0, fixed=False, prob=None)
VariableNode(name=e14, val=0, fixed=False, prob=None)
VariableNode(name=e15, val=0, fixed=False, prob=None)
VariableNode(name=e16, val=0, fixed=True, prob=None)
VariableNode(name=e17, val=0, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(name=exec5_et, val=0, fixed=True, prob=None)
FactorNo

Finally e10 is nominated.

# marginal inference

reset the factor graph

In [8]:
v_e7.unfix()
v_e16.unfix()
factor_graph.reset()
print(factor_graph)

VariableNode(name=e6, val=None, fixed=False, prob=None)
VariableNode(name=e7, val=None, fixed=False, prob=None)
VariableNode(name=e8, val=None, fixed=False, prob=None)
VariableNode(name=e9, val=None, fixed=False, prob=None)
VariableNode(name=e10, val=None, fixed=False, prob=None)
VariableNode(name=e11, val=None, fixed=False, prob=None)
VariableNode(name=e12, val=None, fixed=False, prob=None)
VariableNode(name=e13, val=None, fixed=False, prob=None)
VariableNode(name=e14, val=None, fixed=False, prob=None)
VariableNode(name=e15, val=None, fixed=False, prob=None)
VariableNode(name=e16, val=None, fixed=False, prob=None)
VariableNode(name=e17, val=None, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(name=exec5_et, 

Run marginal inference

In [9]:
factor_graph.marginal_inference_exhaustive(scope)
print(factor_graph)

VariableNode(name=e6, val=None, fixed=False, prob=0.0007054557522498677)
VariableNode(name=e7, val=None, fixed=False, prob=0.5525999148831645)
VariableNode(name=e8, val=None, fixed=False, prob=0.0007054557522498677)
VariableNode(name=e9, val=None, fixed=False, prob=0.0007054557522498677)
VariableNode(name=e10, val=None, fixed=False, prob=0.06495750980594928)
VariableNode(name=e11, val=None, fixed=False, prob=0.0007054557522498682)
VariableNode(name=e12, val=None, fixed=False, prob=0.10141314421407767)
VariableNode(name=e13, val=None, fixed=False, prob=0.0007054557522498682)
VariableNode(name=e14, val=None, fixed=False, prob=0.012892555304154726)
VariableNode(name=e15, val=None, fixed=False, prob=0.0007054557522498678)
VariableNode(name=e16, val=None, fixed=False, prob=0.38366818946756365)
VariableNode(name=e17, val=None, fixed=False, prob=0.0007054557522498679)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNo

order of suspiciousness: [e7, e16, e12, e10, ...]

Q. e12가 3등인 이유가 뭘까?

In [10]:
temporary_variables = [
    VariableNode(name) for name in [var.name for var in factor_graph.variables if not var.name.startswith("exec")]
]
for variable in temporary_variables:
    variable.val = 1 if variable.name in ["e10", "e7"] else 0
for exec_name, result in zip(exec_names, results):
    testnode = VariableNode(exec_name)
    testnode.val = result
    testnode.fix()
    temporary_variables.append(testnode)
for var in temporary_variables:
    print(var)
total_prob = 1
for factor in factor_graph.factors:
    prob = factor.calc_prob(temporary_variables, scope)
    print(prob)
    total_prob *= prob
print(total_prob)

VariableNode(name=e6, val=0, fixed=False, prob=None)
VariableNode(name=e7, val=1, fixed=False, prob=None)
VariableNode(name=e8, val=0, fixed=False, prob=None)
VariableNode(name=e9, val=0, fixed=False, prob=None)
VariableNode(name=e10, val=1, fixed=False, prob=None)
VariableNode(name=e11, val=0, fixed=False, prob=None)
VariableNode(name=e12, val=0, fixed=False, prob=None)
VariableNode(name=e13, val=0, fixed=False, prob=None)
VariableNode(name=e14, val=0, fixed=False, prob=None)
VariableNode(name=e15, val=0, fixed=False, prob=None)
VariableNode(name=e16, val=0, fixed=False, prob=None)
VariableNode(name=e17, val=0, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(name=exec5_et, val=0, fixed=True, prob=None)
0.9863

그 이유는, e12가 문제라기보다, e10이 4등이 될 만큼 별로이기 때문인데, 이는 exec2가 2개이기 때문에 e10을 faulty로 설정하는게 그 만큼 probability에 큰 penalty가 되었기 때문.

~~이를 해결하기 위해서 faulty execution에 적어도 하나의 buggy element가 존재한다는 factor를 넣어주자~~
소용 없다. 어짜피 e12가 올라오는 건 multiple buggy element가 가능하기 때문이라서.

비슷하게 e7을 non faulty로 두게 되면

In [11]:
v_e7.val = 0
v_e7.fix()
factor_graph.marginal_inference_exhaustive(scope)
print(factor_graph)

VariableNode(name=e6, val=None, fixed=False, prob=0.0015641370638418196)
VariableNode(name=e7, val=0, fixed=True, prob=None)
VariableNode(name=e8, val=None, fixed=False, prob=0.0015641370638418196)
VariableNode(name=e9, val=None, fixed=False, prob=0.0015641370638418196)
VariableNode(name=e10, val=None, fixed=False, prob=0.14341099275882224)
VariableNode(name=e11, val=None, fixed=False, prob=0.0015641370638418207)
VariableNode(name=e12, val=None, fixed=False, prob=0.11856478573766392)
VariableNode(name=e13, val=None, fixed=False, prob=0.0015641370638418207)
VariableNode(name=e14, val=None, fixed=False, prob=0.01224226494352411)
VariableNode(name=e15, val=None, fixed=False, prob=0.0015641370638418198)
VariableNode(name=e16, val=None, fixed=False, prob=0.8474901879533238)
VariableNode(name=e17, val=None, fixed=False, prob=0.0015641370638418202)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, 

e16이 가장 faulty하다고 하고, 다시 e16도 non faulty라고 하면

In [12]:
v_e16.val = 0
v_e16.fix()
factor_graph.marginal_inference_exhaustive(scope)
print(factor_graph)

VariableNode(name=e6, val=None, fixed=False, prob=0.01020373633796393)
VariableNode(name=e7, val=0, fixed=True, prob=None)
VariableNode(name=e8, val=None, fixed=False, prob=0.01020373633796393)
VariableNode(name=e9, val=None, fixed=False, prob=0.01020373633796393)
VariableNode(name=e10, val=None, fixed=False, prob=0.9286182279739947)
VariableNode(name=e11, val=None, fixed=False, prob=0.01020373633796393)
VariableNode(name=e12, val=None, fixed=False, prob=0.08757050026577658)
VariableNode(name=e13, val=None, fixed=False, prob=0.010203736337963929)
VariableNode(name=e14, val=None, fixed=False, prob=0.005731340110680901)
VariableNode(name=e15, val=None, fixed=False, prob=0.010203736337963929)
VariableNode(name=e16, val=0, fixed=True, prob=None)
VariableNode(name=e17, val=None, fixed=False, prob=0.010203736337963929)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)

e10이 가장 faulty하게 된다.

# Sumproduct test

In [13]:
v_e7.unfix()
v_e16.unfix()
factor_graph.reset()
print(factor_graph)

VariableNode(name=e6, val=None, fixed=False, prob=None)
VariableNode(name=e7, val=None, fixed=False, prob=None)
VariableNode(name=e8, val=None, fixed=False, prob=None)
VariableNode(name=e9, val=None, fixed=False, prob=None)
VariableNode(name=e10, val=None, fixed=False, prob=None)
VariableNode(name=e11, val=None, fixed=False, prob=None)
VariableNode(name=e12, val=None, fixed=False, prob=None)
VariableNode(name=e13, val=None, fixed=False, prob=None)
VariableNode(name=e14, val=None, fixed=False, prob=None)
VariableNode(name=e15, val=None, fixed=False, prob=None)
VariableNode(name=e16, val=None, fixed=False, prob=None)
VariableNode(name=e17, val=None, fixed=False, prob=None)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, val=0, fixed=True, prob=None)
VariableNode(name=exec2_et_2, val=0, fixed=True, prob=None)
VariableNode(name=exec3_et, val=0, fixed=True, prob=None)
VariableNode(name=exec4_et, val=0, fixed=True, prob=None)
VariableNode(name=exec5_et, 

In [14]:
factor_graph.marginal_sum_product(scope)
print(factor_graph)

iteration 0
iteration 1
iteration 2
iteration 3
Converge after 3 iterations
VariableNode(name=e6, val=1, fixed=False, prob=0.0011642557823892496)
VariableNode(name=e7, val=1, fixed=False, prob=0.6701783378953878)
VariableNode(name=e8, val=1, fixed=False, prob=0.0011642557823892496)
VariableNode(name=e9, val=1, fixed=False, prob=0.0011642557823892496)
VariableNode(name=e10, val=1, fixed=False, prob=0.10145561847692293)
VariableNode(name=e11, val=1, fixed=False, prob=0.0011642557823892503)
VariableNode(name=e12, val=1, fixed=False, prob=0.12373095921070938)
VariableNode(name=e13, val=1, fixed=False, prob=0.0011642557823892496)
VariableNode(name=e14, val=1, fixed=False, prob=0.013419921666754929)
VariableNode(name=e15, val=1, fixed=False, prob=0.0011642557823892498)
VariableNode(name=e16, val=1, fixed=False, prob=0.5160910988349099)
VariableNode(name=e17, val=1, fixed=False, prob=0.0011642557823892503)
VariableNode(name=exec1_et, val=1, fixed=True, prob=None)
VariableNode(name=exec2_et, v